# Given any MILP make Benders' decomposition using provided hints

## Original problem
$$
\begin{align*}
\min_{x,y} \quad & f'y + c'x \\
\mathrm{s.t.} \quad & Ax + By = d \\
& l \leq x \leq u \\
& y \in \{0,1\}
\end{align*}
$$

## Restricted Master Problem
$$
\begin{align*}
\min_{y,z} \quad & z \\
\mathrm{s.t.} \quad & z \geq f'y + (d-By)'w^A_{i,k} + l_k'w^l_{i,k} - u'_k w^u_{i,k}, & i \in \mathrm{OptimalityCuts}, \ k=1,\dots,K \\
& (d_k-By)'w^A_{i,k} + l_k' w^l_{i,k} - u_k' w^u_{i,k} \leq 0, & i \in \mathrm{FeasibilityCuts}, \ k=1,\dots,K
\end{align*}
$$

## Subproblem(k)
$$
\begin{align*}
\min_{x} \quad & f'y^* + c'x  \\
\mathrm{s.t.} \quad & Ax = d - By^* \\
& l \leq x \leq u
\end{align*}
$$

### Dual of subproblem(k)
$$
\begin{align*}
\max_{w^A,w^l,w^u} \quad & (d-By^*)' w^A + l' w^l - u' w^u \\
\mathrm{s.t.} \quad & A' w^A + w^l - w^u = c \\
& w^A \in \mathbb{R}^m \\
& w^l,w^u \geq 0
\end{align*}
$$

In [2]:
from gurobipy import *

In [5]:
import pandas as pd
import numpy as np

UB = 1e3

supply = [10.,30.,40.,20]
demand = [20.,50.,30.]
varcost = [[2.,3.,4.], [3.,2.,1.], [1.,4.,3.], [4.,5.,2.]]
fixedcost = [[10.,30.,20.], [10.,30.,20.], [10.,30.,20.], [10.,30.,20.]]
ns = len(supply)
nd = len(demand)

xup = {(i,j):min(supply[i], demand[j]) for i in range(ns) for j in range(nd)}

In [1]:
def decompose(primal):
    """
    Decompose
    """
    ### Split x and y (cont and binary vars)
    ### Retri

## Original problem

In [280]:
milp = Model('milp')
x0 = {(i,j):milp.addVar(0, UB, 0., GRB.CONTINUOUS, 'x[%d,%d]' % (i,j)) for i in range(ns) for j in range(nd)}
y0 = {(i,j):milp.addVar(0, 1, 0., GRB.BINARY, 'y[%d,%d]'%(i,j)) for i in range(ns) for j in range(nd)}
milp.update()
cons_supply = [milp.addConstr( sum([x0[(i,j)] for j in range(nd)]) <= supply[i])  for i in range(ns)]
cons_demand = [milp.addConstr( sum([x0[(i,j)] for i in range(ns)]) >= demand[j]) for j in range(nd)]
cons_bigM   = [milp.addConstr(  x0[(i,j)] <= xup[(i,j)]*y0[(i,j)]) for i in range(ns) for j in range(nd)]
f = fixedcost
c = varcost
milp.setObjective(sum([f[i][j]*y0[(i,j)] + c[i][j]*x0[(i,j)] for i in range(ns) for j in range(nd)]),
                  GRB.MINIMIZE)

In [281]:
milp.Params.OutputFlag = 0
milp.optimize()

## Split original into RMP and Sub

### Restricted Master Problem
$$
\begin{align*}
\min_{y,z} \quad & z \\
\mathrm{s.t.} \quad & z \geq f'y + (d-By)'w^A_{i,k} + l_k'w^l_{i,k} - u'_k w^u_{i,k}, & i \in \mathrm{OptimalityCuts}, \ k=1,\dots,K \\
& (d_k-By)'w^A_{i,k} + l_k' w^l_{i,k} - u_k' w^u_{i,k} \leq 0, & i \in \mathrm{FeasibilityCuts}, \ k=1,\dots,K
\end{align*}
$$

### Original problem
$$
\begin{align*}
\min_{x,y} \quad & f'y + c'x \\
\mathrm{s.t.} \quad & Ax + By = d \\
& l \leq x \leq u \\
& y \in \{0,1\}
\end{align*}
$$

In [355]:
from scipy.sparse import coo_matrix

def get_expr_coos(expr, var_inds):
    for i in range(expr.size()):
        dvar = expr.getVar(i)
        yield expr.getCoeff(i), var_inds[dvar]

def get_matrix_coo(m, dvars=None):
    """
    Return coeff matrix re-indexed for dvars if provided, else for all variables
    """
    constrs = m.getConstrs()    
    all_vars = m.getVars()
    if dvars is None:
        dvars = m.getVars()    
    var_inds = {v:i for i,v in enumerate(all_vars) if v in dvars}
            
    data = []
    row_inds = []
    col_inds = []
    
    for row_idx, constr in enumerate(constrs):
        for coeff, col_idx in get_expr_coos(m.getRow(constr), var_inds):
            data.append(coeff)
            row_inds.append(row_idx)            
            #col_inds.append(col_idx)
            col_inds.append(dvars.index(all_vars[col_idx]))
    M = len(constrs)
    #N = len(all_vars)
    n = len(dvars)
    #A = coo_matrix((data, (row_inds, col_inds)), (M,N)).tocsr()
    A = coo_matrix((data, (row_inds, col_inds)), (M,n)).tocsr()
    return A #, var_inds

In [361]:
### Subproblem
class Decomposer(object):
    def __init__(self, milp):
        self.milp = milp
        self._INF = 1e6
        self._master = None
        self._sub = None        
        self._d = None
        self._B = None
        self._xs = None
        self._ys = None
#         self._yinds = None
        self._wa = None
        self._wl = None
        self._wu = None
        self._xl = None
        self._xu = None
        self._c  = None
        self._f  = None
        
    def benders_decomp(self):
        """
        TODO: for multiple subproblems.
        """
        master = self.make_master()
        sub = self.make_sub()
        self._master = master
        self._sub = sub
        
        return master, sub
        
    def make_master(self):
        LB = -self._INF
        UB = self._INF
        
        milp = self.milp
        ys0 = [x for x in milp.getVars() if x.VType=='B' or x.VType=='I']
#         B, yinds = get_matrix_coo(milp, dvars=ys0)
        B = get_matrix_coo(milp, dvars=ys0)
        f = [yj.Obj for yj in ys0]
        ny = len(ys0)
        
        master = Model('master')
        z = master.addVar(LB, UB, 0., GRB.CONTINUOUS, 'z')
        ys = [master.addVar(y.LB, y.UB, y.Obj, y.VType, y.VarName) for y in ys0]
        master.addConstr(z >= sum([f[j]*ys[j] for j in range(ny)]))
        master.setObjective(z, GRB.MINIMIZE)
        
        self._B = B
#         self._yinds = yinds
        self._z = z
        self._ys = ys
        self._f = f
        
        return master
        
    def make_sub(self, yopt=None):
        """
        Constraint doesn't change.
        Objective changes with RMP solution, so yopt is optional when initiating.
        """
        milp = self.milp
        xs0 = [x for x in milp.getVars() if x.VType=='C']        
        d = [r.rhs for r in milp.getConstrs()]
        #A,xinds = get_matrix_coo(milp, dvars=xs0)
        A = get_matrix_coo(milp, dvars=xs0)
        
        m = len(d)
        n = len(xs0)
        nx = n
        sub = Model('sub')
        wa = [sub.addVar(LB, UB, 0., GRB.CONTINUOUS, 'wa[%d]'%i) for i in range(m)]
        wl = [sub.addVar(0., UB, 0., GRB.CONTINUOUS, 'wl[%d]'%i) for i in range(n)]
        wu = [sub.addVar(0., UB, 0., GRB.CONTINUOUS, 'wu[%d]'%i) for i in range(n)]
        xl = [x.LB for x in xs]
        xu = [x.UB for x in xs]
        c  = [x.Obj for x in xs]
        
        self._d = d
        self._A = A
        self._xs = xs
        self._xl = xl
        self._xu = xu
        self._c  = c
        self._wa = wa
        self._wl = wl
        self._wu = wu        
        # This dual constraint never changes
        dual_cons = [sub.addConstr(sum([A[i,j]*wa[i] for i in range(m)]) \
                                   + wl[j] - wu[j] == c[j]) for j in range(nx)]
        
        if yopt is not None:
            #sub.setObjective(sum([d[i]-sum([B[i,yinds[yopt[j]]]*yopt[j] for j in range(ny)])*wa[i] for i in range(m)]) +
            sub.setObjective(sum([d[i]-sum([B[i,j]*yopt[j] for j in range(ny)])*wa[i] for i in range(m)]) +
                             sum([xl[j]*wl[j] for j in range(n)]) -
                             sum([xu[j]*wu[j] for j in range(n)]), GRB.MAXIMIZE)
        
        return sub

    def update_subobj(self, yopt):
        sub = self._sub
        if sub is None:
            sub = self.make_sub(yopt)
        
        d = self._d
        B = self._B
#         yinds = self._yinds
        wa = self._wa
        wl = self._wl
        wu = self._wu
        xl = self._xl
        xu = self._xu        
        #sub.setObjective(sum([d[i]-sum([B[i,yinds[yopt[j]]]*yopt[j] for j in range(ny)])*wa[i] for i in range(m)]) +
        sub.setObjective(sum([d[i]-sum([B[i,j]*yopt[j] for j in range(ny)])*wa[i] for i in range(m)]) +
                         sum([xl[j]*wl[j] for j in range(n)]) -
                         sum([xu[j]*wu[j] for j in range(n)]), GRB.MAXIMIZE)

    def make_optcut(self):        
        z = self._z
        f = self._f
        ys = self._ys
        d = self._d
        B = self._B
        xl = self._xl
        xu = self._xu        
        m = len(d)
        n = len(xl)
        
        wa = [w.X for w in self._wa]
        wl = [w.X for w in self._wl]
        wu = [w.X for w in self._wu]
        
        #sum([d[i]-sum([B[i,yinds[ys[j]]]*ys[j] for j in range(ny)])*wa[i] for i in range(m)]) + \
        
        cut = z >= sum([fj*yj for fj,yj in zip(f,ys)]) + \
            sum([d[i]-sum([B[i,j]*ys[j] for j in range(ny)])*wa[i] for i in range(m)]) + \
            sum([xl[j]*wl[j] for j in range(n)]) - \
            sum([xu[j]*wu[j] for j in range(n)])

        return cut

    def make_feascut(self):
        """
        THIS MAY NOT WORK if dvar.X doesn't work for infeas problems.
        Instead, may need FarkasDuals for primal sub problem
        """
        # Get unbounded ray
        wa = [w.X for w in self._wa]
        wl = [w.X for w in self._wl]
        wu = [w.X for w in self._wu]

        ys = self._ys
        d = self._d
        B = self._B
        xl = self._xl
        xu = self._xu
        wa = self._wa
        wl = self._wl
        wu = self._wu
        m = len(d)
        n = len(xl)

        #cut = sum([d[i]-sum([B[i,yinds[ys[j]]]*ys[j] for j in range(ny)])*wa[i] for i in range(m)]) + \
        cut = sum([d[i]-sum([B[i,j]*ys[j] for j in range(ny)])*wa[i] for i in range(m)]) + \
                             sum([xl[j]*wl[j] for j in range(n)]) - \
                             sum([xu[j]*wu[j] for j in range(n)]) <= 0

        return cut
    
    def calc_sub_objval(self, yopt):
        sub = self._sub
        f = self._f
        objval = sub.ObjVal + sum([fj*yj for fj,yj in zip(f,yopt)])
        
        return objval
    
    def get_sub(self):
        if self._sub is None:
            self._sub = self.make_sub()

        return self._sub

    def get_master(self):
        if self._master is None:
            self._master = self.make_master()
            
        return self._master

In [362]:
###==================================================###
### SOLVER-SPECIFIC CALLBACK
### Feas and Opt cuts implemented as lazy constraints
###==================================================###
def benders(model, where):
    GAPTOL = 1e-6
    
    if where in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
        ### Lazy constraints only allowed for MIPNODE or MIPSOL
        ys = decomposer._ys
        z  = decomposer._z        
        if where==GRB.Callback.MIPSOL:
            yopt = [model.cbGetSolution(y) for y in ys]
            zmaster = model.cbGetSolution(z)
        elif where==GRB.Callback.MIPNODE:        
            yopt = [model.cbGetNodeRel(y) for y in ys]
            zmaster = model.cbGetNodeRel(z)
        #************
        print('#'*40)
        print('Updating subproblem objective')
        print('#'*40)
        #************
        decomposer.update_subobj(yopt)        
        sub = decomposer.get_sub()        
        sub.optimize()
        
        if sub.Status == GRB.Status.UNBOUNDED:
            print('#'*40)
            print('Adding feasibility cut')
            print('#'*40)
            # Add feasibility cut, ensuring that cut indeed eliminates current incumbent
            feascut = decomposer.make_feascut(yopt, zmaster)
            model.cbLazy(feascut)            
        else:
            print('#'*40)
            print('Adding optimality cut')
            print('#'*40)
            ### Otherwise add Optimality cut                        
            try:                
                zsub = decomposer.calc_sub_objval(yopt)
                #sub.ObjVal + sum([fj*yj for fj,yj in zip(f,yopt)])
            except:
                zsub = -1e6
            gap = zmaster - zsub
            print('*'*40)    
            print('zSub: %g' % zsub)
            print('zMaster: %g' % zmaster)                        
            print('Gap: %g' % gap)
            print('*'*40)
            # Check for termination
            if abs(gap) > GAPTOL:
                # Need to add optimality cut since master overestimated (for maximization) actual obj
                optcut = decomposer.make_optcut()
                model.cbLazy(optcut)                
            else:
                # Accept as new incumbent
                pass

In [363]:
decomposer = Decomposer(milp)

In [364]:
master,sub = decomposer.benders_decomp()
master.Params.LazyConstraints = 1
master.Params.OutputFlag=0
master.optimize(benders)

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
########################################
Updating subproblem objective
########################################
Optimize a model with 12 rows, 43 columns and 43 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+03, 1e+03]
  Bounds range     [1e+06, 1e+06]
  RHS range        [1e+00, 5e+00]
Presolve removed 12 rows and 43 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  2.000000000e+02
########################################
Adding optimality cut
########################################
****************************************
zSub: 200
zMaster: 0
Gap: -200
****************************************
########################################
Updating subproblem o

In [365]:
master

<gurobi.Model MIP instance master: 1 constrs, 13 vars, Parameter changes: LogFile=gurobi.log, LazyConstraints=1, OutputFlag=0>

In [366]:
milp

<gurobi.Model MIP instance milp: 19 constrs, 24 vars, Parameter changes: LogFile=gurobi.log, OutputFlag=0>

In [367]:
master.ObjVal

200.00000000011642

In [368]:
milp.ObjVal

350.0

In [370]:
B = get_matrix_coo(milp, ys0)

In [371]:
B

<19x12 sparse matrix of type '<type 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

### Restricted Master Problem
$$
\begin{align*}
\min_{y,z} \quad & z \\
\mathrm{s.t.} \quad & z \geq f'y + (d-By)'w^A_{i,k} + l_k'w^l_{i,k} - u'_k w^u_{i,k}, & i \in \mathrm{OptimalityCuts}, \ k=1,\dots,K \\
& (d_k-By)'w^A_{i,k} + l_k' w^l_{i,k} - u_k' w^u_{i,k} \leq 0, & i \in \mathrm{FeasibilityCuts}, \ k=1,\dots,K
\end{align*}
$$

### Dual of subproblem(k)
$$
\begin{align*}
\max_{w^A,w^l,w^u} \quad & (d-By^*)' w^A + l' w^l - u' w^u \\
\mathrm{s.t.} \quad & A' w^A + w^l - w^u = c \\
& w^A \in \mathbb{R}^m \\
& w^l,w^u \geq 0
\end{align*}
$$